In [10]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

In [11]:
# import data
movies = pd.read_csv("/Users/bellehuang/Downloads/STA4101/STA4101-Final-Paper/data/cleaned_movies.csv")

In [12]:
##  Fitting Models 

# LR (dependent variable is tickets.sold without gross)
# Target
movies["tickets_sold"] = pd.to_numeric(movies["tickets_sold"], errors="coerce")

# Splitting the date up
movies["release_date"] = pd.to_datetime(movies["release_date"], errors="coerce")
movies["release_year"]  = movies["release_date"].dt.year
movies["release_month"] = movies["release_date"].dt.month

# Predictors
predictors = ["release_year", "release_month", "Distributor", "Genre", 
              "MPAA","title_has_man", "title_has_love", "title_has_life"]
predictors = [c for c in predictors if c in movies.columns]

# compose X and y
X = movies[predictors].copy()
y = movies["tickets_sold"]

# fit a preliminary model (with all variables available)
formula = (
    "tickets_sold ~ release_year + release_month"
    " +C(Distributor) + C(Genre) + C(MPAA)"
    " + title_has_man + title_has_love + title_has_life"
)

model = smf.ols(formula, data=movies).fit()
print(model.summary())       
pred  = model.predict(movies) 

                            OLS Regression Results                            
Dep. Variable:           tickets_sold   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.292
Method:                 Least Squares   F-statistic:                     5.606
Date:                Sun, 14 Dec 2025   Prob (F-statistic):          4.28e-265
Time:                        12:15:57   Log-Likelihood:            -1.0610e+05
No. Observations:                6335   AIC:                         2.133e+05
Df Residuals:                    5768   BIC:                         2.172e+05
Df Model:                         566                                         
Covariance Type:            nonrobust                                         
                                                                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

In [13]:
# Try variable selection using significance of predictors as criteria
alpha = 0.05  # removal threshold
formula_terms = [
    "release_year", "release_month",
    "C(Distributor)", "C(Genre)", "C(MPAA)",
    "title_has_man", "title_has_love", "title_has_life"
]

def fit_formula(terms):
    f = "tickets_sold ~ " + " + ".join(terms)
    return smf.ols(f, data=movies).fit(), f

# start with full model
model, current_formula = fit_formula(formula_terms)

# iteratively drop worst (highest p) term with p > alpha
while True:
    aov = anova_lm(model, typ=2)
    aov = aov.drop(index="Residual")
    # If all p <= alpha, stop
    worst = aov["PR(>F)"].idxmax()
    worst_p = aov.loc[worst, "PR(>F)"]
    if worst_p <= alpha:
        break

    # Remove that whole term from formula
    if worst in formula_terms:
        formula_terms.remove(worst)

    # Refit
    model, current_formula = fit_formula(formula_terms)

print("Final formula:", current_formula)
print(model.summary())
#removed two predictors but retains the same R^2 and other criteria, stick with this one

/Users/bellehuang/Downloads/STA4101/STA4101-Final-Paper/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 571, but rank is 549
  warnings.warn('covariance of constraints does not have full '
/Users/bellehuang/Downloads/STA4101/STA4101-Final-Paper/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 571, but rank is 549
  warnings.warn('covariance of constraints does not have full '


Final formula: tickets_sold ~ release_year + release_month + C(Distributor) + C(Genre) + C(MPAA) + title_has_man
                            OLS Regression Results                            
Dep. Variable:           tickets_sold   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.292
Method:                 Least Squares   F-statistic:                     5.623
Date:                Sun, 14 Dec 2025   Prob (F-statistic):          1.46e-265
Time:                        12:15:58   Log-Likelihood:            -1.0610e+05
No. Observations:                6335   AIC:                         2.133e+05
Df Residuals:                    5770   BIC:                         2.171e+05
Df Model:                         564                                         
Covariance Type:            nonrobust                                         
                                                                         coef    std err         

/Users/bellehuang/Downloads/STA4101/STA4101-Final-Paper/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 571, but rank is 549
  warnings.warn('covariance of constraints does not have full '


In [14]:
# the descriptive model for predicting tickets_sold only has R^2 = 0.35, not high, try to predict gross with tickets_sold,
# expects better results

# make sure Gross is numeric
movies["Gross"] = pd.to_numeric(movies["Gross"], errors="coerce")

# OLS
formula_gross = (
    "Gross ~ tickets_sold + release_year + release_month"
    " + C(Distributor) + C(Genre) + C(MPAA)"
    " + title_has_man + title_has_love + title_has_life"
)
m_full = smf.ols(formula_gross, data=movies).fit()
print(m_full.summary())

# variable selection
alpha = 0.05
terms = [
    "tickets_sold", "release_year", "release_month",
    "C(Distributor)", "C(Genre)", "C(MPAA)",
    "title_has_man", "title_has_love", "title_has_life"
]

def fit_terms(ts):
    f = "Gross ~ " + " + ".join(ts)
    return smf.ols(f, data=movies).fit(), f

model, current = fit_terms(terms)

while True:
    aov = anova_lm(model, typ=2).drop(index="Residual")
    worst = aov["PR(>F)"].idxmax()
    worst_p = aov.loc[worst, "PR(>F)"]
    if worst_p <= alpha:
        break
    # Remove that term and refit
    terms = [t for t in terms if t != worst]
    model, current = fit_terms(terms)

print("Final formula (p-value backward):", current)
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Gross   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                     1594.
Date:                Sun, 14 Dec 2025   Prob (F-statistic):               0.00
Time:                        12:15:59   Log-Likelihood:            -1.0440e+05
No. Observations:                6335   AIC:                         2.099e+05
Df Residuals:                    5767   BIC:                         2.138e+05
Df Model:                         567                                         
Covariance Type:            nonrobust                                         
                                                                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

/Users/bellehuang/Downloads/STA4101/STA4101-Final-Paper/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 571, but rank is 549
  warnings.warn('covariance of constraints does not have full '
